In [20]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

In [21]:
df = sns.load_dataset("geyser")
df.head()

,duration,waiting,kind
0,3.600,79,long
1,1.800,54,short
2,3.333,74,long
3,2.283,62,short
4,4.533,85,long


In [26]:
X = df.iloc[:,:2]
y = df["kind"]
y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 272 entries, 0 to 271
Series name: kind
Non-Null Count  Dtype 
--------------  ----- 
272 non-null    object
dtypes: object(1)
memory usage: 2.2+ KB


In [27]:
df.columns

Index(['duration', 'waiting', 'kind'], dtype='object')

### Function for train_test_split

In [42]:
def split(X,y):
    from numpy import random
    random.seed(12)
    random_numbers = random.randint(0,len(X),len(X))
    train_index = random_numbers[:int(0.7*len(X))]
    test_index = random_numbers[int(0.7*len(X)):]
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]
    return X_train, X_test,y_train,y_test

In [43]:
X_train, X_test,y_train,y_test= split(X,y)

In [44]:
print(X_train.shape,y_train.shape)
print(X_test.shape, y_test.shape)

(190, 2) (190,)
(82, 2) (82,)


In [45]:
### Preprocessing

In [46]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [47]:
X_train.head()

,duration,waiting
155,4.000,70
253,4.500,73
241,2.350,47
130,1.867,45
259,4.283,79


In [48]:
X_train_trasformed = pd.DataFrame(scaler.fit_transform(X_train),
                                 columns= X_train.columns,
                                 index = X_train.index)
X_train_trasformed.head()

,duration,waiting
155,0.498079,0.008264
253,0.930700,0.222367
241,-0.929571,-1.633196
130,-1.347483,-1.775931
259,0.742943,0.650574


### test preprocessing

In [49]:
X_test_transformed = pd.DataFrame(scaler.transform(X_test),
                                 columns = X_test.columns,
                                 index = X_test.index)
X_test_transformed.head()

,duration,waiting
178,0.498079,1.078781
160,-1.059357,-1.775931
77,0.988672,0.579206
47,-1.145882,-1.204989
223,0.915991,0.365103


### KNN with sklearn

In [50]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train_trasformed, np.ravel(y_train))

KNeighborsClassifier()

In [51]:
y_pred = knn.predict(X_test_transformed)

tem_df = pd.DataFrame({"pred":y_pred,"actual":y_test})
tem_df.head()



,pred,actual
178,long,long
160,short,short
77,long,long
47,short,short
223,long,long


In [53]:


from sklearn import metrics


metrics.accuracy_score(y_test,y_pred)

0.9878048780487805

### KNN from Scratch

In [54]:
class my_knn():
    def __init__(self, k, distance=None ):
        self.k = k
        if distance is None:
            self.distance = self.euclidean_distance
    def euclidean_distance(self, a,b):
        dist = 0
        for i ,j in zip(a,b):
            dist = dist+((i-j)**2)
        dist = dist**0.5
        return dist
    
    def fit(self, X, y):
        self.dataset_ = X.copy()
        self.labels_ = y.copy()
        self.possible_labels_ = np.unique(y)
        
    def predict(self, X):
        predictions = np.zeros(X.shape[0], dtype=int)
        for i in range(X.shape[0]):
            k_nearest = self._find_nearest(X[i])[:self.k]
            indices = [entry[1] for entry in k_nearest]
            k_labels = self.labels_[indices]
            counts = np.bincount(k_labels,
                                 minlength=self.possible_labels_.shape[0])
            pred_label = np.argmax(counts)
            predictions[i] = pred_label
        return predictions
        
    
knn_model = my_knn(k=3)

knn_model.fit(X_train,y_train)

y_pred = knn.predict(X_test_transformed)

tem_df = pd.DataFrame({"actual":y_test,"pred":y_pred})
tem_df




,actual,pred
178,long,long
160,short,short
77,long,long
47,short,short
223,long,long
...,...,...
199,long,long
64,short,short
121,long,long
62,short,short


In [55]:


metrics.accuracy_score(y_test,y_pred)



0.9878048780487805